In [1]:
%load_ext autoreload

%autoreload 2
import imp
import factorilog


In [2]:
%pdb 

Automatic pdb calling has been turned ON


In [40]:


from factorilog import Wire, WireColor

with open("blueprints/sample.bp","r") as file:
    sample = file.read()
    c = factorilog.Layout.fromBlueprint(sample)


In [41]:
netlist_meta = c.toNetString(meta=True)
print(netlist_meta)

c: medium-electric-pole        | 1
b <= c: 1 black if green > 2   | 2
a <= c: red = green + 0        | 3
d <= b, e: @ all if black = 0  | 4
d <= a, b: @ red if black = 1  | 5
d: medium-electric-pole        | 6
e <=: 5 red                    | 7
||
1 | -5 -2 
2 | -2.5 -2 S
3 | -2.5 0 S
4 | 0.5 0 S
5 | 0.5 -1 S
6 | 4 -1 
7 | -3 1 S
a | red 5i-3o
b | green 2o-5i 5i-4i
c | red 3i-1 1-2i
d | green 5o-6 6-4o
e | red 7-4i


In [42]:
layout = factorilog.Layout.fromNetlist(netlist_meta)

{
  "Entities": [
    {
      "Descriptor": {
        "name": "medium-electric-pole"
      },
      "ID": 1,
      "nets": {
        "pass": [
          "c"
        ],
        "out": null
      }
    },
    {
      "Descriptor": {
        "name": "decider-combinator",
        "behavior": {
          "decider_conditions": {
            "output_signal": {
              "type": "virtual",
              "name": "signal-black"
            },
            "copy_count_from_input": false,
            "comparator": ">",
            "first_signal": {
              "type": "virtual",
              "name": "signal-green"
            },
            "constant": 2
          }
        }
      },
      "ID": 2,
      "nets": {
        "out": [
          "b"
        ],
        "in": [
          "c"
        ]
      }
    },
    {
      "Descriptor": {
        "name": "arithmetic-combinator",
        "behavior": {
          "arithmetic_conditions": {
            "output_signal": {
              "type": "vi

In [43]:
for hyp in layout.hyperwires:
    print(hyp.name)
        

d
a
b
c
e


In [44]:
print(layout.toNetString(meta=True))

c: medium-electric-pole        | 1
b <= c: 1 black if green > 2   | 2
a <= c: red = green + 0        | 3
d <= e, b: @ all if black = 0  | 4
d <= a, b: @ red if black = 1  | 5
d: medium-electric-pole        | 6
e <=: 5 red                    | 7
||
1 | -5 -2 
2 | -2.5 -2 S
3 | -2.5 0 S
4 | 0.5 0 S
5 | 0.5 -1 S
6 | 4 -1 
7 | -3 1 S
d | green 6-4o 5o-6
a | red 5i-3o
b | green 4i-5i 2o-5i
c | red 2i-1 3i-1
e | red 7-4i


In [35]:
#wires per term
l_ents = list(c.ents)
for ent in l_ents:
    print(ent.name,len(ent.terminals))
    for term in ent.terminals:
        print("\t",len(term.wires))
        

decider-combinator 2
	 3
	 1
medium-electric-pole 1
	 2
decider-combinator 2
	 1
	 1
constant-combinator 1
	 1
medium-electric-pole 1
	 2
arithmetic-combinator 2
	 1
	 1
decider-combinator 2
	 2
	 1


In [20]:
ar = l_ents[1]
con = c.getConnectedTerminals(ar.terminals[0],WireColor.red)
print(con)
print(list(con)[0].ent.name)


{<factorilog.Terminal object at 0x7f2a64029710>, <factorilog.Terminal object at 0x7f2a64029550>}
constant-combinator


In [21]:
net = c.getNetlist()
print(len(net))
for hyp in list(net):
    print(hyp.color, len(hyp.terminals))

5
WireColor.red 3
WireColor.green 3
WireColor.green 3
WireColor.red 2
WireColor.red 2


In [22]:
w=l_ents[1].terminals[0].wires
for w1 in l_ents[1].terminals[0].wires:
    print(w1.color)
    print(w1.terminals)

WireColor.green
frozenset({<factorilog.Terminal object at 0x7f2a64029550>, <factorilog.Terminal object at 0x7f2a640295f8>})
WireColor.red
frozenset({<factorilog.Terminal object at 0x7f2a64029710>, <factorilog.Terminal object at 0x7f2a64029550>})


In [23]:
#num wires
wires = set()
for ent in list(c.ents):
    for term in ent.terminals:
        wires.update(term.wires)
print(len(wires))

8


In [26]:

import pprint
from slpp import slpp as lua
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(lua.decode(sample))

[ { 'connections': { '1': { 'red': [ {'circuit_id': 1, 'entity_id': 2},
                                     {'circuit_id': 1, 'entity_id': 3}]}},
    'entity_number': 1,
    'name': 'medium-electric-pole',
    'position': {'x': -5, 'y': -2}},
  { 'connections': { '1': {'red': [{'entity_id': 1}]},
                     '2': {'green': [{'circuit_id': 1, 'entity_id': 5}]}},
    'control_behavior': { 'decider_conditions': { 'comparator': '>',
                                                  'constant': 2,
                                                  'copy_count_from_input': False,
                                                  'first_signal': { 'name': 'signal-green',
                                                                    'type': 'virtual'},
                                                  'output_signal': { 'name': 'signal-black',
                                                                     'type': 'virtual'}}},
    'direction': 2,
    'entity_number': 2,
  

In [284]:
import test_parser
import json
imp.reload(test_parser)

text=\
"""84"""

parser = test_parser.TestParser(parseinfo=False)
ast = parser.parse(text, rule_name='start')
print(ast)
print(json.dumps(ast, indent=2)) # ASTs are JSON-friendy

['84']
[
  "84"
]
